In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.callbacks import *
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
nltk.download('words')
nltk.download('stopwords')
%matplotlib inline

# 전처리
#### 전처리 실행한 모델과
#### 아무것도 전처리하지 않은 모델 간 성능비교할 것

In [ ]:
train = pd.read_csv('eda_data.csv')
print(train.head())
input = train['html']
target = train['label']

# bert_layer : Bert 모델 층 로드
* 로드할 기본 Bert모델도 성능이 좋아
* 추가로 많은 층을 쌓을 필요가 없음
# vocab_file : Bert 모델 내 단어 로드
* Bert모델은 우리 데이터셋 내 단어가 자신들의 단어사전에 포함되어 있는지 확인하고
    * 없다면 그 단어를 하위 단어로 분할해서 확인하는 과정을 반복한다
    * 최종적으로는 알파벳 단위로까지 분할해서 학습 --> 은어들도 학습은 가능

In [ ]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1",
                           trainable=True)   
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()  

# Tensorflow의 Bert모델 관련 모듈 다운로드

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
# 데이터 전처리 함수
import tokenization
tk = tokenization.FullTokenizer(vocab_file)   # Bert용 토크나이저 --> 단어를 음절단위로 구분

def bert_encode(html,tk,max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    for text in html:
        text = tk.tokenize(text)   # 토크나이징
        text = text[:max_len-2]  
        # 단어 맨 뒤 2개 안가져오는 이유? 빈칸을 만들어놓고 special token을 넣기 위함
        # special token을 통해 어떤 학습을 할지 Bert 모델이 결정함
        input_sequence = ["[CLS]"] + text + ["[SEP]"]    # 문장 앞뒤로 special token을 넣어줌
        # cls --> 분류모델용 special token
        # sep --> 문장간 구분자 역할
        pad_len = max_len-len(input_sequence)  # max_len 보다 html 짧은 경우 남는 공간      
        tokens = tk.convert_tokens_to_ids(input_sequence)    # 각 단어를 숫자 임베딩
        tokens += [0] * pad_len      # pad_len은 0을 부여할 것
        all_tokens.append(tokens)    # all_tokens에는 전체 html이 존재
        
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        # mask : 모델에게 어디서부터 어디까지 봐야하는지 알려주는 기능
        # 어디서부터 어디까지가 실제 문장이고 의미없는 패딩인지 구분하는 기능
        all_masks.append(pad_masks)
        
        segment_ids = [0] * max_len      
        # 각 단어가 어떤 문장에 포함되어 있는지 모델에게 알려주는 기능
        # 보통 단일문장은[1], 다중문장은[0] 값을 부여한다고 함
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)
    # array형태로 바꿔주어야 학습가능

train_input = bert_encode(input.values,tk,max_len=110)  # max_len은 변경가능!

# 결과물
* all_tokens = [[CLS], 80, 90, 222.......[SEP], [CLS], 20, 80, 1110, ....[SEP], .......]
* all_masks = [1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0, 1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,....]
* all_segments = [0,0,0,0,0,0,0,.....0,0,0,0]

In [ ]:
# 모델링(input 요소가 3개이므로 input층도 3개)

def build_model(bert_layer,max_len=512):
    input_word_ids = Input(shape=(max_len),dtype=tf.int32,name="input_word_ids") # name은 반드시 고정!
    input_mask = Input(shape=(max_len),dtype=tf.int32,name="input_mask")
    input_segment_ids = Input(shape=(max_len),dtype=tf.int32,name="segment_ids")
    _,sequence_output = bert_layer([input_word_ids,input_mask,input_segment_ids]) 
    clf_output = sequence_output[:,0,:]  
    # :(데이터 batch--> 모든 데이터 처리) , 0(데이터셋의 모든 단어를 사용하겠다는 뜻) , :(각 단어 임베딩)
    output = Dense(2,activation='softmax')(clf_output)
    model = Model(inputs=[input_word_ids,input_mask,input_segment_ids],outputs=output)
    model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer=Adam(lr=0.00002))
    return model

model = build_model(bert_layer,max_len=110)
model.summary()

In [ ]:
# 모델 학습

cp = callbacks.ModelCheckpoint('best-bert-model.h5')
es = callbacks.EarlyStopping(patience=3,restore_best_weights=True,verbose=1)
rl = callbacks.ReduceLROnPlateau(patience=2)
history_2 = model.fit(train_input,pd.get_dummies(target),batch_size=32,epochs=5,validation_split=0.2,callbacks=[cp,es,rl])

# 채점
#### train 데이터와 동일한 전처리 실행
#### 위의 bert_encode 함수에 데이터 삽입

In [ ]:
test = pd.read_csv('test_data.csv')
input_2 = test['html']
test_input = bert_encode(input_2.values,tk,max_len=110)  # max_len은 변경가능!

In [ ]:
# 위 학습에서 가장 좋은 결과가 나온 옵션 로드
bert_model = models.load_model('best-bert-model.h5',custom_objects={'KerasLayer':hub.KerasLayer})
bert_model.evaluate(test_input,pd.get_dummies(test['label']))

In [ ]:
result = bert_model.predict(test_input)
print(result)
print('---------------------------------------정답 확인---------------------------------------')
print(result.argmax(1))

In [ ]:
df_res = pd.DataFrame(result.argmax(1))
df_res.to_csv('bert_check.csv',index=False)

# 결론
#### 장점 
* 비교적 짧은 시간내 정확한 학습 가능
* 모델을 거의 그대로 가져오기 때문에 복잡하게 층을 여러개 쌓을필요 X
* 버리는 단어 없이 거의 모든 단어 학습가능
#### 단점
* 학습할 html 길이를 110보다 높게 설정시 RAM 터지는 현상발생
* max_len 높일시(최소 500은 필요) 학습시간 예상 불가능
